# Examen Parcial EJERCICIO 2 - Clasificación multiclase ONE vs ALL

## Introduction

En este ejercicio se implementa la regresion one-vs-all a un conjunto de datos de MNIST FGSM, Un conjunto de datos similar a MNIST de 70.000 imágenes de 28x28 etiquetadas como método de signo de gradiente rápido. MNIST-FGSM es un conjunto de datos de imágenes adversas que consta de un conjunto de entrenamiento de 60.000 ejemplos y un conjunto de prueba de 10.000 ejemplos. Cada ejemplo es una imagen en escala de grises de 28x28, asociada con una etiqueta de 10 clases: (0,1, 2, 3, 4, 5, 6, 7, 8, 9).
Cada fila es una imagen separada.
La columna 1 es la etiqueta de clase (enteros del 0 al 9).
Las columnas restantes son números de píxeles (784 en total).
Cada valor es la oscuridad del píxel (1 a 255)

*   Nombre Dataset: **MNIST FGSM**
*   Url Dataset: https://www.kaggle.com/datasets/sudulakishore/mnist-fgsm?select=mnist_train.csv


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# utilizado para la manipulación de directorios y rutas
import os

# Cálculo científico y vectorial para python
import numpy as np

# Libreria para graficos
from matplotlib import pyplot

# Modulo de optimizacion en scipy
from scipy import optimize

# modulo para preprocesamiento del Dataset
import pandas as pd
# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

In [3]:
# Cargar el dataset
data = pd.read_csv('/content/drive/MyDrive/SIS 420 INTELIGENCIA ARTIFICAL 2024/1er Parcial/D2 MNIST FGSM ONEVSALL/MNIST FGSM/mnist_train.csv', delimiter=',')


In [4]:
data.shape
data.dtypes

label    int64
0        int64
1        int64
2        int64
3        int64
         ...  
779      int64
780      int64
781      int64
782      int64
783      int64
Length: 785, dtype: object

In [5]:
dataset = data.copy()

In [6]:
# La entrada es de 25 caracteristicas + un elemento contando con x0
input_layer_size  = 785

#Etiquetas del los numeros del 0 al 9
num_labels = 10

X = dataset.iloc[:, 1:]  # Selecciona todas las filas y todas las columnas desde la segunda hasta la última
y = dataset.iloc[:, 0]    # Selecciona todas las filas y la primera columna

y[y == 10] = 0
print(X)
print(y)

m = y.size

        0   1   2   3   4   5   6   7   8   9  ...  774  775  776  777  778  \
0      25  25  25  24  24  25  25  25  25  17  ...   16   24    3   25   20   
1       1  24  25  25  25  25  25  25  20   0  ...   25    5   23    0    0   
2       0   0   0   0   0   0   0   0   5  25  ...   16   25   25   25   20   
3      25  25  25  25  25  25  25  25  25  25  ...    0    0    0    0    0   
4       0   0   0  11   8   0   0   1  18   1  ...   28    5    0    0    0   
...    ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ...  ...  ...  ...  ...   
59995  25  25  25  12   0   0   0   0   5  23  ...    1   20   25   25   20   
59996  25  25  25  25  25  25  25  25  20   0  ...    0   20   25   25   20   
59997  25  25  25  12   0   1   1   1   1  10  ...    0    0    1    1   20   
59998   0   5  24  25  10   0   0   1   1   1  ...    9   20    3    1   20   
59999  25  25  25  24   8   0   0  21   5  25  ...    0    0    0    0    0   

       779  780  781  782  783  
0        0    0   

In [7]:
def  featureNormalize(X):
    X_norm = X.copy()
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    mu = np.mean(X, axis = 0)
    sigma = np.std(X, axis = 0)
    X_norm = (X - mu) / sigma

    return X_norm, mu, sigma

In [8]:
# llama featureNormalize con los datos cargados
X_norm, mu, sigma = featureNormalize(X)

In [9]:
# Configurar la matriz adecuadamente, y agregar una columna de unos que corresponde al termino de intercepción.
m, n = X.shape
# Agraga el termino de intercepción a A
# X = np.concatenate([np.ones((m, 1)), X_norm], axis=1)
X = X_norm
# X = np.concatenate([np.ones((m, 1)), X], axis=1)

In [10]:
def sigmoid(z):

    return 1.0 / (1.0 + np.exp(-z))

In [11]:
def lrCostFunction(theta, X, y, lambda_):

#     alpha = 0.003
#     theta = theta.copy()
    # Inicializa algunos valores utiles
    m = y.size

    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)

    J = 0
    grad = np.zeros(theta.shape)

    h = sigmoid(X.dot(theta.T))

    temp = theta
    temp[0] = 0

#     J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h)))
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))

    grad = (1 / m) * (h - y).dot(X)
#     theta = theta - (alpha / m) * (h - y).dot(X)
    grad = grad + (lambda_ / m) * temp

    return J, grad
#    return J, theta

In [12]:
def oneVsAll(X, y, num_labels, lambda_):
    # algunas variables utiles
    m, n = X.shape

    all_theta = np.zeros((num_labels, n + 1))

    # Agrega unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 100}
        res = optimize.minimize(lrCostFunction,
                                initial_theta,
                                (X, (y == c), lambda_),
                                jac=True,
                                method='CG',
                                options=options)

        all_theta[c] = res.x

    return all_theta

In [13]:
lambda_ = 0.1
all_theta = oneVsAll(X, y, num_labels, lambda_)
print(all_theta.shape)

(10, 785)


In [14]:
print(all_theta)

[[-1.28750739e+01  5.68947575e-01 -4.67631590e-01 ...  3.18127656e-01
  -2.14917585e-01 -7.78911220e-03]
 [-1.56737945e+01 -6.69717276e-01 -1.50297035e-01 ...  1.85436535e-02
   3.03839443e-01 -2.50660722e-01]
 [-7.22093670e+00  4.38534896e-01  3.16194841e-02 ... -6.31199661e-02
  -3.98792141e-02  1.49008709e-02]
 ...
 [-1.03552200e+01  6.45861549e-02 -1.50378324e-01 ... -8.79493540e-02
   6.00744439e-02 -2.12203599e-01]
 [-7.53627025e+00 -3.67414450e-01  4.03655259e-01 ...  5.25788884e-02
   1.22525897e-01 -1.07801630e-01]
 [-7.65438705e+00  8.02763928e-02  2.91295158e-01 ...  1.65181599e-01
  -1.06137901e-01  2.13705367e-02]]


In [15]:
def predictOneVsAll(all_theta, X):

    m = X.shape[0];
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # Add ones to the X data matrix
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1)

    return p

In [16]:
print(X.shape)
pred = predictOneVsAll(all_theta, X)
print('Precision del conjuto de entrenamiento: {:.2f}%'.format(np.mean(pred == y) * 100))


(60000, 784)
Precision del conjuto de entrenamiento: 96.13%


In [20]:
#Evaluar con el dataset de test: train.csv
# Cargar el dataset
data_t = pd.read_csv('/content/drive/MyDrive/SIS 420 INTELIGENCIA ARTIFICAL 2024/1er Parcial/D2 MNIST FGSM ONEVSALL/MNIST FGSM/mnist_test.csv', delimiter=',')

dataset_t = data_t.copy()

# Verificar el tipo de datos
print(dataset_t.dtypes)

label    int64
0        int64
1        int64
2        int64
3        int64
         ...  
779      int64
780      int64
781      int64
782      int64
783      int64
Length: 785, dtype: object


In [21]:
# La entrada es de 25 caracteristicas + un elemento contando con x0
input_layer_size  = 785

#Etiquetas del numero (de 0 a 9)
num_labels = 10

X_test = dataset_t.iloc[:, 1:]  # Selecciona todas las filas y todas las columnas excepto la última
y_test = dataset_t.iloc[:, 0]    # Selecciona todas las filas y la última columna

y_test[y_test == 10] = 0
print(X_test)
print(y_test)

m = y_test.size

       0   1   2   3   4   5   6   7   8   9  ...  774  775  776  777  778  \
0      0   0   0  12  10  20  25  25  20   1  ...   25   25   25   25   20   
1      0   0   0  11  24  25  25  25  25  25  ...    8   18   23    1    5   
2     23  23  23  24  25  25  25  25  25  16  ...   25    5    0    0    0   
3     25  25  25  13   1   0   0   0   0   0  ...    0    0    0    0    0   
4     25  25  25  12  16   5   0   0   0   0  ...    0    0    0   25   25   
...   ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ...  ...  ...  ...  ...   
9995  25  25  25  12  16   5   0   0   0   0  ...   15   23   23   23   23   
9996  23  18   0  11  24  25  25   1   0   0  ...    1   18    1    1    0   
9997  25  25  25  25   9   0   0   0   0   9  ...   24    5    0    0    0   
9998  25  25  25  12  15   5   0   0   5  16  ...   23   25   25   25   20   
9999   0   0   0  11   8  20  25  25  18   0  ...   25   25    1    0    0   

      779  780  781  782  783  
0       0    0    0    0    0  

In [22]:
# Imprimir la forma de X_test
print(X_test.shape)

#Normalizamos
X_norm, mu, sigma = featureNormalize(X_test)

# Configurar la matriz adecuadamente, y agregar una columna de unos que corresponde al termino de intercepción.
m, n = X_test.shape
X_test = X_norm
# X = np.concatenate([np.ones((m, 1)), X], axis=1)

# Realizar predicciones en todo el conjunto de datos X_test
pred = predictOneVsAll(all_theta, X_test)

# Calcular y mostrar la precisión del conjunto de prueba
print('Precisión del conjunto de prueba: {:.2f}%'.format(np.mean(pred == y_test) * 100))

# Imprimir los primeros 10 valores predichos y las etiquetas originales correspondientes del conjunto de prueba
#print("Predicciones para los primeros 10 ejemplos:", pred[:-10:])
#print("Etiquetas originales del test.csv", y_test)

# Imprimir todos los valores predichos del conjunto de prueba
print("Predicciones para todos los ejemplos:", pred)

(10000, 784)
Precisión del conjunto de prueba: 94.77%
Predicciones para todos los ejemplos: [7 2 1 ... 4 5 6]


Con Datos de ENTRENAMIENTO se consigue una precisión del 96.13% en un problema de clasificación de diez clases es un resultado gratificante que expresa que esta bien aplicado los procedimientos en el modelo.

Con Datos de TEST se consigue una precisión del 94.77% en un problema de clasificación de diez clases es un resultado gratificante al igual que del modelo de entrenamiento. Porque comparando con los datos originales y coiniciden de manera eficaz del metodo Hay algunas conclusiones y consideraciones en el metodo realizado que podemos mencionar:

--El modelo de Mnist de una imagen de 28x28 pixeles lo que da lugar a
784 pixeles en total.
--El dataset contiene 785 caracteristicas la 1ra columna contiene al numero (y a predecir o clasificar) y las demas 784 columnas a cada uno de los pixeles de la imagen. Cada valor es la oscuridad del píxel (1 a 255).
--El dataset en cuestion no tiene datos inconsistentes ni columnas irrelevantes, esta completamente limpio.

 Podriamos a llegar a concluir que el modelo en este sentido se aplica de la mejor manera y con una precision superior al 90% tanto entrenamiento y test.